In [7]:
import pandas as pd
import geopandas as gpd
import os
from joblib import Parallel, delayed
from pathlib import Path
from proxi_API.data.settings import N_CORES, H3_ZOOM
from proxi_API.model.data_processing import get_city, get_streets
from proxi_API.model import data_aggregation
from proxi_API.model import mobility_indices
from proxi_API.model import h3_mapping
import osmnx as ox
import logging
import h3
from shapely import Polygon

In [214]:
place = "Barcelona, Spain"
G = ox.graph_from_place(place, network_type="walk")

In [235]:
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False, node_geometry= False).reset_index()[['x','y']]
nodes = nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [237]:
# Step 2: Extract node coordinates
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False, node_geometry= False).reset_index()[['x','y']]
nodes = nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

resolution = 9
nodes["h3_index"] = nodes.apply(lambda row: h3.latlng_to_cell(row["lat"], row["lon"], resolution), axis=1)

h3_polygons = {
    h3_index: Polygon(h3.cell_to_boundary(h3_index))
    for h3_index in nodes["h3_index"].unique()
}

h3_gdf = gpd.GeoDataFrame(h3_polygons.items(), columns=["h3_index", "geometry"])
h3_gdf['lon'] = h3_gdf.geometry.centroid.x
h3_gdf['lat'] = h3_gdf.geometry.centroid.y
h3_gdf['nearest_node'] = ox.distance.nearest_nodes(G, X = h3_gdf['lon'], Y = h3_gdf['lat'])

In [238]:
# Flatten the categories dictionary into a single mapping (amenity -> category)
amenity_to_category = {amenity: category for category, amenities in categories.items() for amenity in amenities}

def compute_length(Id1, Id2, G):
    path = ox.routing.shortest_path(G, Id1, Id2)
    if path is None:
        result = -1
    
    else:
        result = sum(G[u][v][0]["length"] for u, v in zip(path[:-1], path[1:]))

    return result


In [298]:
def proximity_time(p1, category, G, N = 100):
    i = 1
    while True:
        pois = ox.features_from_point((p1.lon, p1.lat), dist = N*i, tags = {'amenity':True})
        pois['category'] = pois['amenity'].map(amenity_to_category)
        pois = pois[pois['category'] == category]
        pois = pois.to_crs(epsg=3857)
        pois["nearest_node"] = ox.distance.nearest_nodes(G, X = pois.geometry.centroid.x, Y = pois.geometry.centroid.y)
        pois['distance'] = pois['nearest_node'].apply(lambda x: compute_length(p1.nearest_node, x, G))
        
        pois = pois[pois['distance']!= -1]
        
        if len(pois)>0:
            break

        i += 1
    
    dis = pois['distance'].min()


    return dis
        

In [299]:
h3_gdf = h3_gdf
h3_gdf['distance'] = h3_gdf.apply(lambda row: proximity_time(row, 'restaurant', G), axis=1)

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

np.float64(13372.69694671751)

In [ ]:
def compute_length(Id1, Id2, G):
    path = ox.routing.shortest_path(G, Id1, Id2)
    return sum(G[u][v][0]["length"] for u, v in zip(path[:-1], path[1:]))

In [105]:
def distance_to_ammenity(p_origin, df, category, G = G):
    df = df[df['category'] == category].copy()
    df['distance'] = df['ID'].apply(lambda x: compute_length(p_origin, x, G))

    return df['distance'].min()


In [11]:
pois.amenity.unique()

array(['shower', 'bicycle_rental', 'bicycle_parking', 'toilets',
       'motorcycle_parking', 'dive_centre', 'bar'], dtype=object)

In [29]:
pois

amenity    category   node_value
element id                                                      
node    3643911945               shower    services   3643911945
        4374272505       bicycle_rental     outdoor   4374272505
        4668532006               shower    services   4668532006
        5262549963       bicycle_rental     outdoor   5262549963
        6933505402      bicycle_parking     outdoor   6933505402
        9658016746              toilets    services   9658016746
        9699830617      bicycle_parking     outdoor   9699830617
        9700065742   motorcycle_parking   transport   9700065742
        9700065743   motorcycle_parking   transport   9700065743
        12018261473         dive_centre     outdoor  12018261473
        12020145685                 bar  restaurant  12020145685
        12020145686                 bar  restaurant  12020145686

In [125]:
h3_gdf

AttributeError: 'NoneType' object has no attribute 'copy'

,h3_index,geometry
0,89394461cb7ffff,"POLYGON ((41.37647 2.14848, 41.3747 2.14881, 4..."
1,89394461993ffff,"POLYGON ((41.36495 2.17954, 41.36317 2.17987, ..."
2,89394460a53ffff,"POLYGON ((41.37157 2.18658, 41.36979 2.18691, ..."
3,89394460acfffff,"POLYGON ((41.37455 2.1881, 41.37277 2.18843, 4..."
4,89394460acbffff,"POLYGON ((41.37391 2.18408, 41.37213 2.18441, ..."
...,...,...
1082,8939446185bffff,"POLYGON ((41.34318 2.14638, 41.3414 2.14671, 4..."
1083,8939446184bffff,"POLYGON ((41.34083 2.14888, 41.33905 2.14921, ..."
1084,89394460807ffff,"POLYGON ((41.39738 2.21076, 41.39561 2.21109, ..."
1085,89394461bc7ffff,"POLYGON ((41.33571 2.16841, 41.33393 2.16874, ..."


In [141]:
categories = {
    'culture': [
        'arts_centre', 'cinema', 'exhibition_centre', 'theatre', 'library',
        'museum'
    ],
    'education': [
        'beauty_school', 'college', 'community_centre', 'coworking_space',
        'dance_school', 'driving_school', 'kindergarten', 'language_school',
        'music_school', 'prep_school', 'school', 'university'
    ],
    'healthcare': [
        'clinic', 'dentist', 'doctors', 'hospital', 'nursing_home', 'pharmacy',
        'veterinary'
    ],
    'outdoor': [
        'bench', 'bicycle_parking', 'bicycle_rental', 'bicycle_repair_station',
        'fountain', 'park', 'playground', 'shelter', 'table', 'tap',
        'waste_basket', 'waste_disposal', 'water_point', 'shower', 'toilets', 'dive_centre'
    ],
    'physical': [
        'dojo', 'fitness_centre', 'gym', 'sports_centre', 'stadium'
    ],
    'restaurant': [
        'bar', 'cafe', 'fast_food', 'food_court', 'ice_cream', 'pub',
        'restaurant'
    ],
    'services': [
        'aparthotel', 'apartments', 'bank', 'bureau_de_change',
        'car_rental', 'car_wash', 'charging_station', 'courthouse',
        'crematorium', 'events_venue', 'funeral_home', 'gambling',
        'internet_cafe', 'love_hotel', 'marketplace', 'money_transfer',
        'office', 'parcel_locker', 'parking', 'parking_entrance',
        'parking_space', 'police', 'post_box', 'post_depot', 'post_office',
        'public_building', 'recycling', 'research_institute',
        'stock_exchange', 'taxi', 'telephone', 'townhall', 'vehicle_inspection', 'toilets', 'shower'
    ],
    'supplies': [
        'atm', 'locker', 'luggage_locker', 'photo_booth', 'public_bookcase',
        'toy_library', 'vending_machine'
    ],
    'transport': [
        'boat_rental', 'bus_station', 'fuel', 'motorcycle_parking',
        'motorcycle_rental', 'train_station'
    ],
    'others': [
        'brothel', 'fixme', 'flight_attendant', 'stripclub', 'swingers club'
    ]
}


In [13]:
import json

In [15]:
with open('src/proxi_API/data/POIs/categories.json', 'w') as fp:
    json.dump(categories, fp)

In [70]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import h3
from shapely.geometry import Polygon

# Load the graph
place = "Barcelona, Spain"
G = ox.graph_from_place(place, network_type="walk", retain_all=True)

# Ensure G is valid before conversion
if G is None or len(G.nodes) == 0:
    raise ValueError("Error: The graph G is empty or not loaded properly.")

# Convert graph to GeoDataFrame
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)

# Ensure nodes DataFrame is valid
if nodes is None or nodes.empty:
    raise ValueError("Error: No nodes found in the graph.")

# Generate H3 indexes
resolution = 9
nodes["h3_index"] = nodes.apply(lambda row: h3.latlng_to_cell(row["y"], row["x"], resolution), axis=1)

# Generate H3 polygons
h3_polygons = {
    h3_index: Polygon(h3.cell_to_boundary(h3_index, geo_json=True))
    for h3_index in nodes["h3_index"].unique()
}

# Convert to GeoDataFrame
h3_gdf = gpd.GeoDataFrame(pd.DataFrame(h3_polygons.items(), columns=["h3_index", "geometry"]))

# Print sample output
print(h3_gdf.head())


TypeError: cell_to_boundary() got an unexpected keyword argument 'geo_json'

In [71]:
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)

In [103]:
nodes[['geometry']].type.unique()

array(['Point'], dtype=object)

In [122]:
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False, node_geometry= False).reset_index()[['x','y']]

In [123]:
nodes

,x,y
0,2.149443,41.375385
1,2.182977,41.363454
2,2.183127,41.363479
3,2.187593,41.370365
4,2.187686,41.370169
...,...,...
220342,2.183651,41.383894
220343,2.183191,41.383623
220344,2.183386,41.383880
220345,2.164346,41.390094


In [88]:
nodes.keys()

Index(['y', 'x', 'street_count', 'highway', 'ref', 'railway', 'junction',
       'geometry'],
      dtype='object')

In [84]:
nodes = ox.graph_to_gdfs(G, nodes=True, edges=False, fill_edge_geometry=False)

if nodes is None:
    raise ValueError("Error: graph_to_gdfs() returned None, possibly due to an empty graph.")


In [85]:
if nodes.empty:
    raise ValueError("Error: The nodes GeoDataFrame is empty. No nodes found in the graph.")

In [132]:
import pandas as pd
import geopandas as gpd

pd.set_option("display.max_columns", None)
gpd.options.display_precision = 5

display(h3_gdf)  # Instead of print(h3_gdf)


AttributeError: 'NoneType' object has no attribute 'copy'

,h3_index,geometry
0,89394461cb7ffff,"POLYGON ((41.37647 2.14848, 41.3747 2.14881, 4..."
1,89394461993ffff,"POLYGON ((41.36495 2.17954, 41.36317 2.17987, ..."
2,89394460a53ffff,"POLYGON ((41.37157 2.18658, 41.36979 2.18691, ..."
3,89394460acfffff,"POLYGON ((41.37455 2.1881, 41.37277 2.18843, 4..."
4,89394460acbffff,"POLYGON ((41.37391 2.18408, 41.37213 2.18441, ..."
...,...,...
1082,8939446185bffff,"POLYGON ((41.34318 2.14638, 41.3414 2.14671, 4..."
1083,8939446184bffff,"POLYGON ((41.34083 2.14888, 41.33905 2.14921, ..."
1084,89394460807ffff,"POLYGON ((41.39738 2.21076, 41.39561 2.21109, ..."
1085,89394461bc7ffff,"POLYGON ((41.33571 2.16841, 41.33393 2.16874, ..."


In [133]:
print(h3_gdf.head())
h3_gdf.to_csv("debug_output.csv")

          h3_index                                           geometry
0  89394461cb7ffff  POLYGON ((41.37647 2.14848, 41.3747 2.14881, 4...
1  89394461993ffff  POLYGON ((41.36495 2.17954, 41.36317 2.17987, ...
2  89394460a53ffff  POLYGON ((41.37157 2.18658, 41.36979 2.18691, ...
3  89394460acfffff  POLYGON ((41.37455 2.1881, 41.37277 2.18843, 4...
4  89394460acbffff  POLYGON ((41.37391 2.18408, 41.37213 2.18441, ...
